In [1]:
# 1) pick dataset
# 2) make n_split_runs=3 runs of 5-kfold
# 3) for n_model_runs=3 
# 4) train an ensemble of n_ens=5 models
# 5) for each single-nn model make n_ue_runs=5 UE runs
# 6) for EUE make a UE run
# 7) for ensemble-UE make n_ue_runs=5 UE runs
# 8) save data for a future

In [2]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2


In [3]:
sys.path = [ '/home/etsymbalov/.local/lib/python3.6/site-packages'] + sys.path

In [4]:
import os
from pathlib import Path
import random

import matplotlib.pyplot as plt
import pickle
import torch
import numpy as np

# from experiment_setup import get_model, set_random, build_estimator
from alpaca.model.ensemble import MLPEnsemble
from alpaca.dataloader.builder import build_dataset
from experiments.utils.data import scale, multiple_kfold

plt.rcParams['figure.facecolor'] = 'white'

In [5]:
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.cuda.set_device(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



In [6]:
config = {
    'nn_runs': 100,
    'runs': 2,
    'max_runs': 20,
    # Our train config
   'layers': [8, 128, 128, 64, 1],
#     'layers': [8, 256, 256, 128, 1],
    #'layers': [8, 2048, 2048, 1024, 1],
    'epochs': 10_000,
    'validation_step': 100,
    
    # Our train config
    'nll_layers': [8, 256, 256, 128, 2],
    'nll_epochs': 400,
    'nll_validation_step': 50,
    
    'acc_percentile': 0.1,
    'patience': 10,
    'dropout_rate': 0.2,
    'dropout_uq': 0.5,
    'batch_size': 256,
    'dataset': 'kin8nm',
    'l2_reg': 1e-5,
    'ood_percentile': 90,
    
    'optimizer': {'type': 'Adam', 'lr': 0.01, 'weight_decay':1e-5},
    'n_split_runs': 3,
    'n_model_runs': 3,
    'n_ens': 5,
    'n_ue_runs': 5,
    'k_folds': 10,
    'verbose': False,
}


In [7]:
train_opts = ['patience', 'dropout_rate', 'epochs', 'batch_size', 'validation_step', 'verbose']
config['train_opts'] = {k: config[k] for k in config if k in train_opts}


In [8]:
datasets = [
    'boston_housing', 'concrete', 'energy_efficiency',
    'kin8nm', 'naval_propulsion', 'ccpp', 'red_wine', 
    'yacht_hydrodynamics'
]

In [9]:
for dataset_name in datasets:
    dataset = build_dataset(dataset_name, val_split=0.0) 
    x_set, y_set = dataset.dataset('train')
    print(dataset_name, x_set.shape)
    config['layers'][0] = x_set.shape[-1]

boston_housing (506, 13)
concrete (1030, 8)
energy_efficiency (768, 8)
kin8nm (8192, 8)
naval_propulsion (11934, 16)
ccpp (9568, 4)
red_wine (1599, 11)
yacht_hydrodynamics (308, 6)


In [ ]:
# over datasets
for dataset_name in datasets:
    dataset = build_dataset(dataset_name, val_split=0.0) 
    x_set, y_set = dataset.dataset('train')
    config['layers'][0] = x_set.shape[-1]
    # over different splits
    for split_cnt in range(config['n_split_runs']):
        kfold_iterator = multiple_kfold(config['k_folds'], 
                                        len(x_set), 
                                        config['k_folds'])
        # within one split
        for kfold_cnt, (val_idx, train_idx) in enumerate(kfold_iterator): 
            # MIND THE ORDER
            x_train, y_train = x_set[train_idx], y_set[train_idx]
            x_val, y_val = x_set[val_idx], y_set[val_idx]
            x_train, x_val, x_scaler = scale(x_train, x_val)
            y_train, y_val, y_scaler = scale(y_train, y_val)
            
            train_opts = config['train_opts'].copy()
            # over model runs
            for model_cnt in range(config['n_split_runs']):
                model = MLPEnsemble(config['layers'], 
                                    n_models=config['n_ens'], 
                                    reduction='mean')
                model.fit((x_train, y_train),
                          (x_val, y_val),
                          **train_opts)
                
                fname = f'{dataset_name[:4]}_split={split_cnt}_kfold={kfold_cnt}_model={model_cnt}'
                dct = {
                    'config': config,
                    'state_dict': model.state_dict(),
                    'data': (x_train, y_train, x_val, y_val, x_scaler, y_scaler)
                }
                dir = Path(os.getcwd()) / 'data' / 'regression'
                with open(dir / f'{fname}.pickle', 'wb') as f:
                    pickle.dump(dct, f)

In [ ]:
x_train.shape, x_val.shape